## Using TextFile

<br>Get the consolidated account statement  from CAMS
<br>Remove  Password  Protection using print to PDF
<br>Open with PHantom Foxit PDF or Chrome and do  select all, copy and paste to text file

In [1]:
import pandas as pd
import logging

import src.constants as ct
from src.parsedata import DataParser
from src.initialize import Logger

pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)



## Read and Parse MF Transaction Data from text file

In [2]:
# %%time

# logger = Logger()
# logger.initilaze_logging(display_logs = 'yes') 

# file_path = 'data\consol_trans_18_June_2021.txt'
file_path = 'data\consol_trans_18_June_2021_chrome.txt'
dp = DataParser()
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_path)

In [3]:
mf_trans_df.groupby('scheme_name').agg({'scheme_name':'count'})


,scheme_name
scheme_name,
DSP Small Cap Fund-Direct Plan-Growth,7
Franklin India Flexi Cap Fund-Direct-Growth,56
Franklin India Low Duration Fund-Direct-Growth,1
Franklin India Smaller Companies Fund-Direct-Growth,21
Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth,1
HDFC Top 100 Fund-Direct Plan-Growth Option,14
ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,2
ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth,3
ICICI Prudential Long Term Equity Fund-Direct Plan-Growth,2


In [4]:
mf_trans_df[-30:]

,scheme_name,trans_date,amount,units,unit_price
199,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-05-21,25000.00,450.013,55.554
200,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-06-21,25000.00,439.090,56.936
201,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-07-22,25000.00,452.096,55.298
202,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-08-21,25000.00,464.037,53.875
203,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-09-23,25000.00,432.331,57.826
204,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-10-22,25000.00,433.975,57.607
205,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-11-21,25000.00,416.354,60.045
206,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2019-12-23,25000.00,409.614,61.033
207,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2020-01-21,25000.00,397.994,62.815
208,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2020-02-24,25000.00,398.521,62.732


In [5]:
mf_hdr_df

,scheme_name,latest_nav,latest_nav_date
0,DSP Small Cap Fund-Direct Plan-Growth,98.2220,2021-06-17
1,Franklin India Flexi Cap Fund-Direct-Growth,903.5355,2021-06-17
2,Franklin India Smaller Companies Fund-Direct-Growth,83.3396,2021-06-17
3,Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth,30.2922,2021-06-17
4,Franklin India Low Duration Fund-Direct-Growth,24.2658,2021-06-17
5,HDFC Top 100 Fund-Direct Plan-Growth Option,661.4190,2021-06-17
6,ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,29.7814,2021-06-17
7,ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth,26.0015,2021-06-17
8,ICICI Prudential Long Term Equity Fund-Direct Plan-Growth,562.5800,2021-06-17
9,ICICI Prudential Nifty Next 50 Index Fund-Direct Plan-Growth,35.1068,2021-06-17


In [6]:

# t1= '117TSD1G-Mirae Asset Tax Saver Fund - Direct Growth Registrar :'

# scheme_name = 'Mirae Asset Tax Saver Fund-Direct Growth'

# scheme_words = scheme_name.split('-')
# if scheme_words[-1] == 'Direct Growth':
#     scheme_words[-1] = 'Direct-Growth'
#     scheme_name = '-'.join([text for text in  scheme_words])  
# scheme_name 

## Calculate MF Capital Gains

In [7]:
%%time
from src.capitalgains import CapitalGains
target_ltcg = 100000
cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)

Wall time: 274 ms


In [8]:
cg.output_hdr_df

,scheme_name,latest_nav,latest_nav_date,gf_nav,amount,units,ltcg,stcg,current_amt,perc_gain,target_ltcg,target_units,comments
0,DSP Small Cap Fund-Direct Plan-Growth,98.2220,17-Jun-2021,70.1630,759990.00,13196.649,295479.48,61869.86,1296201.26,70.56,100000.00,3563.917,Sell 3563.92 units for LTCG of 100000.0
1,Franklin India Flexi Cap Fund-Direct-Growth,903.5355,17-Jun-2021,637.6120,920000.00,1667.943,483450.65,0.00,1507045.76,63.81,100000.00,376.051,Sell 376.05 units for LTCG of 100000.0
2,Franklin India Smaller Companies Fund-Direct-Growth,83.3396,17-Jun-2021,65.5900,697000.00,12301.352,291192.24,13758.60,1025189.77,47.09,100000.00,5582.729,Sell 5582.73 units for LTCG of 100000.0
3,Franklin India Low Duration Fund-Direct-Growth,24.2658,17-Jun-2021,19.9766,300000.00,15952.610,68423.93,0.00,387102.84,29.03,68423.93,15952.610,Total LTCG 68423.93 is less than target LTCG 100000: Sell 15952.61 units for LTCG of 68423.93
4,HDFC Top 100 Fund-Direct Plan-Growth Option,661.4190,17-Jun-2021,498.0290,569833.57,1331.043,219706.64,0.00,880377.13,54.50,100000.00,612.031,Sell 612.03 units for LTCG of 100000.0
5,ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,29.7814,17-Jun-2021,21.8368,699980.00,29838.991,129766.37,2220.24,888646.93,26.95,100000.00,12587.168,Sell 12587.17 units for LTCG of 100000.0
6,ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth,26.0015,17-Jun-2021,19.8749,799975.00,36754.760,106882.29,2091.77,955678.90,19.46,100000.00,16322.264,Sell 16322.26 units for LTCG of 100000.0
7,ICICI Prudential Nifty Next 50 Index Fund-Direct Plan-Growth,35.1068,17-Jun-2021,27.5101,577500.00,22713.467,221485.09,0.00,797397.13,38.08,100000.00,11887.388,Sell 11887.39 units for LTCG of 100000.0
8,ICICI Prudential US Bluechip Equity Fund-Direct Plan-Growth,47.9900,17-Jun-2021,25.0600,599970.00,14699.403,0.00,105454.33,705424.35,17.58,0.00,0.000,No LTCG exists hence calculation is not applicable
9,ICICI Prudential Value Discovery Fund-Direct Plan-Growth,230.3200,17-Jun-2021,157.6400,979979.00,6474.229,309577.68,108583.71,1491144.43,52.16,100000.00,1375.894,Sell 1375.89 units for LTCG of 100000.0


In [9]:
cg.output_trans_df.head()

,scheme_name,trans_date,amount,units,unit_price,latest_nav,latest_nav_date,gf_nav,new_purch_nav,ltcg,stcg,cumil_ltcg,cumil_units,current_amt,perc_gain
0,DSP Small Cap Fund-Direct Plan-Growth,05-Jan-2017,60000.0,1141.770,52.550,98.222,17-Jun-2021,70.163,70.163,32036.92,0.0,32036.92,1141.77,112146.93,86.91
1,DSP Small Cap Fund-Direct Plan-Growth,10-Jan-2017,100000.0,1895.842,52.747,98.222,17-Jun-2021,70.163,70.163,53195.43,0.0,85232.36,3037.61,186213.39,86.21
2,DSP Small Cap Fund-Direct Plan-Growth,12-Jan-2017,100000.0,1877.511,53.262,98.222,17-Jun-2021,70.163,70.163,52681.08,0.0,137913.44,4915.12,184412.89,84.41
3,DSP Small Cap Fund-Direct Plan-Growth,13-Jan-2017,100000.0,1880.795,53.169,98.222,17-Jun-2021,70.163,70.163,52773.23,0.0,190686.66,6795.92,184735.45,84.74
4,DSP Small Cap Fund-Direct Plan-Growth,16-Jan-2017,100000.0,1875.891,53.308,98.222,17-Jun-2021,70.163,70.163,52635.63,0.0,243322.29,8671.81,184253.77,84.25


## Using PDF File
<br> Issue: some trnasctions are converted to garbage text

In [10]:
import pdfplumber
import pdfminer
import logging
import re

from typing import Iterator
from  datetime import datetime
from collections import defaultdict
from src.utils import is_valid_date


class PasswordError(Exception):
         pass

class PDFParser:
    def __init__(self):
        pass
    def _read_data_from_file(self,file_path:str, password = '')->list[str]:
        '''
        Read the PDF transaction file from CAMS + FTAMIL        
        '''

        try:
         pdf = pdfplumber.open(file_path, password = password)
        except pdfminer.pdfdocument.PDFPasswordIncorrect as ex:
            if not password:
                msg = f'File {file_path} is Password Protected, please provide a password'                 
            else:
                msg = f'Incorrect Password, please provide correct password'      

            logging.error(msg)  
            ex.args = (msg,) 
            raise
           

           

        text_all = ''
        for page in  pdf.pages:
            text = page.extract_text()
            text_all = text_all + text 

        pdf.close()

        mf_data_list = text_all.split('\n')
        return mf_data_list        


    def _generate_batch(self, mf_data_list:list[str] )->Iterator[list[str]]:
        """
        yeild batches for data corresponding to a single Mutual Fund    
        A MF transaction will start with keyword PAN, hence create a batch of
        data between two instances of keyword 'PAN'
        """
        for i, _ in enumerate(mf_data_list):
         if mf_data_list[i].startswith('Folio No'):
           first_idx = i
           break

        for i in range(first_idx+1, len(mf_data_list)):
            if mf_data_list[i].startswith('Folio No'):
                yield mf_data_list[first_idx:i+1]
                first_idx = i

        # For last batch
        yield mf_data_list[first_idx:]

    def _get_latest_nav(self, line:str)->tuple[float, datetime]:
        line = line.replace(':', '').replace(',', '')  
        latest_nav_data = line.split()
        latest_nav_date = latest_nav_data[2]
        latest_nav_date = datetime.strptime(latest_nav_date, ct.DATE_FORMAT)
        latest_nav = float(latest_nav_data[4])
        return  latest_nav,   latest_nav_date

    def _get_numerical_data(self,line:str)->tuple[float, float, float]:
        nums = line.split()[-4:]
        nums= [value.replace(',', '') for value in nums]
        # all(isinstance(x,float) for x in amt ), amt
        try:
            nums= [float(value) for value in nums]
        except ValueError:
            raise ValueError(f'Invalid data in {nums} while parsing numerical data')
        amount = nums[0]
        
        units  = nums[1]
        unit_price=  nums[2]
        return amount, units, unit_price

    def _parse_mf_scheme_name(self,mf_data:list[str])->str:  

        #MF scheme name can be split in multiple lines.
        scheme_name = ''
        for text in mf_data[1:]:
            if text[:13] != ' Opening Unit':
                scheme_name = scheme_name + text
            else:
                break
    

        #Remove brackets and texts inside brackets
        # Input: 'D788-DSP Small Cap Fund - Direct Plan - Growth (formerly DSP Micro Cap Fund)(Advisor: DIRECT) Registrar : CAMS'	
        # Output:'D788-DSP Small Cap Fund - Direct Plan - Growth Registrar : CAMS'
        scheme_name= re.sub(r'\(.*?\)\ *', '', scheme_name)

        # input : 'D788-DSP Small Cap Fund - Direct Plan - Growth Registrar : CAMS'
        # output: 'D788-DSP Small Cap Fund - Direct Plan - Growth'
        scheme_name = scheme_name.split('Registrar')[0].strip()

        # input: 'D788-DSP Small Cap Fund - Direct Plan - Growth'
        # output:'DSP Small Cap Fund-Direct Plan-Growth'
        scheme_name =  '-'.join([text.strip() for text in scheme_name.split('-')[1:]])     

        #Convert GROWTH to Camelcase (Growth)
        # input :'4859909139674 Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-GROWTH'
        # output: '4859909139674 Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth'
        scheme_words = scheme_name.split('-')
        scheme_words[-1] = scheme_words[-1].title()
        scheme_name = '-'.join([text for text in  scheme_words])     

        #Remove leading digits for funds like Franklin
        # input:'4859909139674 Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth'
        # output: 'Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth'
        if scheme_name .split()[0].isnumeric():
            scheme_name = ' '.join([text for text in scheme_name .split()[1:]])

        return scheme_name  

    def parse_mf_data(self,file_path:str, password= '' )->tuple[pd.DataFrame,pd.DataFrame]:
        '''
        Read the text file having MF tranactions from CAMS+FTAMIL
        Generate Batches for single MF scheme and parse MF data
        Transaction data and Latest NAV will be parsed from text file
        '''
        mf_data_dict =defaultdict(list)
        mf_nav_dict =defaultdict(list)

        mf_data_list = self._read_data_from_file(file_path,password)

        # loop over a batch having data for single MF Scheme
        for mf_data in self._generate_batch(mf_data_list):
            
            mf_scheme = self._parse_mf_scheme_name(mf_data)
            logging.debug(f'Parsing data for MF {mf_scheme}')
            
            for line in mf_data:
                date_text = line[:11]

                #only process lines which start with date
                if is_valid_date(date_text,  ct.DATE_FORMAT ):
                
                    try:
                        amount, units, unit_price = self._get_numerical_data(line)                
                    except ValueError:
                        continue
                    else:                  
                        mf_data_dict['scheme_name'].append(mf_scheme) 
                        trans_date = datetime.strptime(date_text, ct.DATE_FORMAT)
                        mf_data_dict['trans_date'].append(trans_date)
                        mf_data_dict['amount'].append(amount)
                        mf_data_dict['units'].append(units)
                        mf_data_dict['unit_price'].append(unit_price)

                #Latest Nav and Date in separate dict      
                if line[:6] == 'NAV on':        
                    latest_nav, latest_nav_date =   self._get_latest_nav(line)

                    #For segregated Portfolios the NAV will be zero, we will ignore them
                    if latest_nav != 0:
                        mf_nav_dict['scheme_name'].append(mf_scheme)
                        mf_nav_dict['latest_nav'].append(latest_nav)
                        mf_nav_dict['latest_nav_date'].append(latest_nav_date)
                        logging.debug(f'Latest Nav:{latest_nav}, Nav Date:{latest_nav}')
                    else:
                        logging.debug(f'Latest Nav:{latest_nav}, MF Scheme{mf_scheme} will be ignored')


        mf_trans_df = pd.DataFrame.from_dict(mf_data_dict)       
        mf_hdr_df = pd.DataFrame.from_dict(mf_nav_dict)
        logging.debug(f'Number of Mutual Funds Scheme: {mf_hdr_df.shape[0]}\n')
        return  mf_trans_df, mf_hdr_df

In [11]:
# logger = Logger()
# logger.initilaze_logging(display_logs = 'no') 

parser = PDFParser()
pdf_file_path = 'data\consol_trans_18_June_2021_pass.pdf'
# pdf_file_path = 'data\consol_trans_18_June_2021.pdf'

password = 'qwerty123'


mf_data_list = parser._read_data_from_file(pdf_file_path,password)
mf_trans_df, mf_hdr_df = parser.parse_mf_data(pdf_file_path, password = password )

# gen = parser._generate_batch(mf_data_list )
# for batch in gen:
#     scheme_name = parser._parse_mf_scheme_name(batch)
#     print(scheme_name)


In [12]:
mf_trans_df.groupby('scheme_name').agg({'scheme_name':'count'})

,scheme_name
scheme_name,
DSP Small Cap Fund-Direct Plan-Growth,7
Franklin India Flexi Cap Fund-Direct-Growth,48
Franklin India Low Duration Fund-Direct-Growth,1
Franklin India Smaller Companies Fund-Direct-Growth,21
Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth,1
HDFC Top 100 Fund-Direct Plan-Growth Option,14
ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,3
ICICI Prudential Long Term Equity Fund-Direct Plan-Growth,2
ICICI Prudential Nifty Next 50 Index Fund-Direct Plan-Growth,28


In [13]:
mf_hdr_df 

""
